In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from titanic import credentials, transformations, models
from titanic.storage import Storage

In [3]:
import mlflow

mlflow.set_tracking_uri('http://127.0.0.1:5001')
mlflow.sklearn.autolog()

## Load titanic training dataset

In [4]:
credentials.load()
storage = Storage(bucket = "industrializing-ml-models")

In [5]:
df_train = storage.read_csv('titanic/train.csv')
df_train = transformations.add_all_features(df_train)

### Train classifier

In [6]:
train_features = transformations.select_features_for_ml(df_train)
train_target = transformations.select_target_for_ml(df_train)
classifier, accuracy = models.fit_random_forest(train_features, train_target)

print(f"Classifier accuracy: {accuracy}")

2023/11/30 22:12:48 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '381280a3b3394e1da7fe224d88b44ffc', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


Classifier accuracy: 0.9233055315885341


In [7]:
mlflow.sklearn.log_model(
    sk_model=classifier,
    artifact_path="sklearn-model",
    signature=mlflow.models.infer_signature(train_features, train_target),
    registered_model_name="titanic-random-forest",
)

Successfully registered model 'titanic-random-forest'.
2023/11/30 22:12:51 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: titanic-random-forest, version 1
Created version '1' of model 'titanic-random-forest'.


In [8]:
mlflow.end_run()